# Монады Maybe, Either, [], Writer, State. Примеры их использования

## Maybe

Частично определённые функции – это такие функции, которые определены не для всех значений аргументов. Примером такой функции может быть функция поиска предыдущего числа для натуральных чисел.
Поскольку числа натуральные, то для нуля такого числа нет. Для описания этого поведения мы можем воспользоваться специальным типом Maybe.

In [1]:
data Maybe a = Nothing | Just a
    deriving (Show, Eq, Ord)

Частично определённая функция имеет тип a -> Maybe b (рис. 6.2), если всё в порядке и значение было
вычислено, она вернёт (Just a), а в случае ошибки будет возвращено значение Nothing. Теперь мы можем
определить нашу функцию так:

In [19]:
pred :: Nat -> Maybe Nat
pred Zero = Nothing
pred (Succ a) = Just a

Для Zero предыдущий элемент не определён.
Значение функции pred завёрнуто в упаковку Maybe, и для того чтобы воспользоваться им нам придётся
разворачивать его каждый раз. Как будет выглядеть функция извлечения дважды предыдущего натурального
числа:

In [20]:
pred2 :: Nat -> Maybe Nat
pred2 x =
    case pred x of
    Just (Succ a) -> Just a
    _ -> Nothing

In [21]:
pred2 (Succ Zero)

Nothing

http://hackage.haskell.org/package/base-4.14.0.0/docs/Data-Maybe.html экзамплы функций из класса Maybe

## Either

Рассмотрим ещё один стандартный тип. Он определён в Prelude. Это тип Either (дословно – один из
двух). Этот тип принимает два параметра:

In [5]:
data Either a b = Left a | Right b


Значение может быть либо значением типа a, либо значением типа b. Часто этот тип используют как
Maybe с информацией об ошибке. Конструктор Left хранит сообщение об ошибке, а конструктор Right
значение, если его удалось вычислить.

In [6]:
headSafe :: [a] -> Either String a
headSafe [] = Left "Empty list"
headSafe (x:_) = Right x

http://hackage.haskell.org/package/base-4.14.0.0/docs/Data-Either.html 
экзамплы функций из класса Either

## []

Объявим в таком стиле списки:


In [ ]:
data [a] where
[] :: [a]
(:) :: a -> [a] -> [a]

Конструктор пустого списка [] является константой, а конструктор объединения элемента со списком
(:), является функцией. Типы определяют примитивы и методы составления из примитивов, некоторые конструкторы по сути являются константами, а другие функциями. Эти “методы” определяют базовые значения типа, все другие значения будут комбинациями базовых.

Функции-накопители при вычислении за ширмой накапливают некоторое значение. Функция-накопитель
похожа на функцию с состоянием но без стрелки, по которой состояние подаётся в функцию (рис. 6.10).
Функция-накопитель имеет тип: a -> (b, msg)

## Writer

http://hackage.haskell.org/package/mtl-2.2.2/docs/Control-Monad-Writer-Class.html

In [52]:
newtype Writer msg b = Writer (b, msg)

Значения типа msg мы будем называть сообщениями. Смысл функций a -> Writer msg b заключается
в том, что при вычислении они накапливают в значении msg какую-нибудь информацию. Это могут быть
отладочные сообщения. Или база данных, которая открыта для всех функций на запись.

In [ ]:
half :: Int -> Writer String Int
half x = do
        tell ("I just halved " ++ (show x) ++ "!")
        return (x `div` 2)

Простой пример логера входных данных, вызвав дважды half увидим

In [ ]:
runWriter $ half 8 >>= half
=> (2, "I just halved 8!I just halved 4!")

## State

В Haskell нельзя изменять значения. Новые сложные значения описываются в терминах базовых значений. Но как же тогда мы сможем описать функцию с состоянием? Функцию, которая принимает на вход
значение, составляет результат на основе внутреннего состояния и значения аргумента и обновляет состояние. Поскольку мы не можем изменять состояние единственное, что нам остаётся – это принимать значение
состояния на вход вместе с аргументом и возвращать обновлённое состояние на выходе. 

newtype State s a = State (s -> (a, s)) 

In [73]:
import Control.Monad.State.Lazy

In [ ]:
greeter :: State String String
greeter = do
    name <- get
    put "newState"
    return ("hello, " ++ name ++ "!")

Функция runState просто извлекает функцию из оболочки State.

In [64]:
runState greeter "currentstate"

("hello, currentstate!","newState")

In [47]:
data Nat = Zero | Succ Nat
    deriving (Show, Eq, Ord)